In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import scipy.stats as stats
import matplotlib.gridspec as gridspec
import seaborn as sns
import pygplates
from gprm import ReconstructionModel

# Note, here we load in the functions from the .py file (in the same folder as this notebook)
import my_functions as myfunc

%matplotlib inline
%load_ext autoreload
%autoreload 2

pygmt-session [WARNING]: Representation of font type not recognized. Using default.
pygmt-session [WARNING]: Representation of font type not recognized. Using default.
pygmt-session [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.
begin [WARNING]: Representation of font type not recognized. Using default.


In [2]:
from gprm.datasets import Reconstructions
Y2019 = Reconstructions.fetch_Young2019()

In [3]:
# Define an ordered dictionary that will define the colour coding of each environment type
from collections import OrderedDict 
palette = OrderedDict()
palette['A'] = 'red' 
palette['B'] = 'blue' 
palette['C'] = 'green'

markersize = 60
reconstruction_model = Y2019

In [ ]:
# import the 'PointDistributionOnaSphere' class
from gprm.GPlatesReconstructionModel import PointDistributionOnSphere
import pygmt
# generate the evenly distributed points that we will use to define the 'bins' for the spatial binning
# Note that the number N must be a power of 2. The higher the number, the denser the points
equal_area_points = PointDistributionOnSphere(distribution_type='healpix',N=16)

In [5]:
import get_distances_for_dataframe as gdfd

# This function will take a set of partitioned points, reconstruct them back to a specified time,
# then determine the number of points within each spatial bin based on the bin centers created above
def get_binned_distances(partitioned_df, reconstruction_model, reconstruction_time, anchor_plate_id=0):
    rlo,rla,rd = myfunc.distance_to_point_features(partitioned_df, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    bin_counts = equal_area_points.point_feature_heatmap(
        [pygplates.PointOnSphere(point) for point in zip(rla,rlo)])
    return bin_counts

def wedges_to_gmt2(x,y,wedge_fractions,wedge_colors,legends):
    row_sums = np.sum(wedge_fractions,axis=1)
    normed_rows = np.divide(wedge_fractions,row_sums[:,np.newaxis])
    cumulative_angles = np.hstack((np.zeros((len(x),1)),np.cumsum(normed_rows, axis=1))) * 360.
    for row in range(len(x)):
        for i,wedge_color in enumerate(wedge_colors):
            start_angle = cumulative_angles[row,i]
            stop_angle = cumulative_angles[row,i+1]
            #myfunc.write_netcdf_grid('tmp.nc',x[row],y[row],start_angle,stop_angle)
            #pt.write_xyz_file('tmp.xyz',zip(x,y,start_angle,stop_angle))
            with open('tmp.xyz', 'w') as output_file:
                output_file.write(' '.join(str(item) for item in (x[row],y[row],start_angle,stop_angle)) + '\n')
            fig.plot(data='tmp.xyz', style='w0.10c', color=wedge_color, label=legends[i], t='20')
        #fig.legend(spec=None, position='JTR+jTR+o0.2c', box='+gwhite+p0.2p')  
        
def wedges_to_gmt3(x,y,wedge_fractions,wedge_colors,legends):
    row_sums = np.sum(wedge_fractions,axis=1)
    normed_rows = np.divide(wedge_fractions,row_sums[:,np.newaxis])
    cumulative_angles = np.hstack((np.zeros((len(x),1)),np.cumsum(normed_rows, axis=1))) * 360.
    for row in range(len(x)):
        for i,wedge_color in enumerate(wedge_colors):
            start_angle = cumulative_angles[row,i]
            stop_angle = cumulative_angles[row,i+1]
            #myfunc.write_netcdf_grid('tmp.nc',x[row],y[row],start_angle,stop_angle)
            #pt.write_xyz_file('tmp.xyz',zip(x,y,start_angle,stop_angle))
            with open('tmp.xyz', 'w') as output_file:
                output_file.write(' '.join(str(item) for item in (x[row],y[row],start_angle,stop_angle)) + '\n')
            fig.plot(data='tmp.xyz', style='w0.10c', color=wedge_color, label=legends[i], transparency='20')
        

In [6]:
# Load in some test data from the database
df_ABC = pd.read_excel('data/DZDB4.xlsx',sheet_name="Samples")

df_ABC.loc[:, 'FROMAGE'] = '0'
df_ABC.loc[:, 'TOAGE'] = '0'

special_number = np.linspace(50, 400, num=8, endpoint=True, retstep=False, dtype=None)
print(special_number)
for i in range(len(df_ABC['FROMAGE'])):
    if df_ABC.loc[i,('Est. Depos. Age (Ma)')] in special_number:
         df_ABC.loc[i,('FROMAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] + 49.99
         df_ABC.loc[i,('TOAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] - 50
    else:
        df_ABC.loc[i,('FROMAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] + 50
        df_ABC.loc[i,('TOAGE')] = df_ABC.loc[i,('Est. Depos. Age (Ma)')] - 50

# set some columns to fit the GPlates naming convention
#df_ABC['FROMAGE'] = pbdb.FROMAGE
#df_ABC['TOAGE'] = pbdb.TOAGE
df_ABC.loc[:, 'longitude'] = df_ABC.loc[:, 'Longitude']
df_ABC.loc[:, 'latitude'] = df_ABC.loc[:, 'Latitude']
df_ABC.loc[:, 'environment'] = df_ABC.loc[:, 'type']

# use the functions in the 'myfunc' collection to create different sets of features with
# plate ids assigned
# in each case, the set of features are those where the word(s) in the 'environment' column 
# of the df_ABC table matches a given string
partitioned_A = myfunc.create_partitioned_feature_selection(df_ABC, 'A', Y2019)
partitioned_B = myfunc.create_partitioned_feature_selection(df_ABC, 'B', Y2019)
partitioned_C = myfunc.create_partitioned_feature_selection(df_ABC, 'C', Y2019)

[ 50. 100. 150. 200. 250. 300. 350. 400.]


In [9]:

def fig6(pattern_color1, pattern_color2, pattern_t, pen_color1, pen_color2, anchor_plate_id=0, region='d', frame='f', projection= 'W8c'):
    fig.basemap(region=region, projection=projection, frame=frame)
    reconstruction_time = 50.

    reconstructed_continents = Y2019.polygon_snapshot('continents', reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_plates = Y2019.plate_snapshot(reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_continents.plot(fig, color='Gainsboro',pen='gray')

    A_bin_counts = get_binned_distances(partitioned_A, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    B_bin_counts = get_binned_distances(partitioned_B, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    C_bin_counts = get_binned_distances(partitioned_C, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)

    ratios2 = np.empty(shape=[0, 3])
    lonr = []
    latr = []
    for i in range(equal_area_points.latitude.shape[0]):
        ratios = np.vstack([A_bin_counts[i],B_bin_counts[i],C_bin_counts[i]]).T
        if np.any(ratios>0):
            ratios2 = np.append(ratios2, ratios, axis=0)
            lonr.append(equal_area_points.longitude[i])
            latr.append(equal_area_points.latitude[i])
    fig.plot(x=lonr, y=latr, pen='white', style="c0.11c")
    wedges_to_gmt2(lonr,latr,ratios2,['darkred','darkblue','darkgreen'],['A','B','C'])

    fig.basemap(region=region, projection=projection, frame=frame)
    fig.basemap(region=[0, 8, 0, 4], projection='X8c/4c', frame=True, transparency="100")
    fig.text(x=0.4, y=3.6, text=str(int(reconstruction_time))+" Ma", font=["7p,Helvetica",'black'],justify=["LM"])


    fig.shift_origin(xshift='8.1c')    
    fig.basemap(region=region, projection=projection, frame=frame)
    reconstruction_time = 150.

    reconstructed_continents = Y2019.polygon_snapshot('continents', reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_plates = Y2019.plate_snapshot(reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_continents.plot(fig, color='Gainsboro',pen='gray')

    A_bin_counts = get_binned_distances(partitioned_A, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    B_bin_counts = get_binned_distances(partitioned_B, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    C_bin_counts = get_binned_distances(partitioned_C, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    fig.plot(data="data/additional_pologys/polygon_1501_red.gmt",color=pattern_color1, pen=pen_color1, transparency=pattern_t)
    #fig.plot(data="data/additional_pologys/polygon_1501_red.gmt",color="p11+fdarkred+b-", pen="2p,darkred", t=60)
    fig.plot(data="data/additional_pologys/polygon_1502_green.gmt",color=pattern_color2, pen=pen_color2, transparency=pattern_t)
    #fig.plot(data="data/additional_pologys/polygon_1502_green.gmt",color="p11+fdarkgreen+b-", pen="2p,darkgreen", t=60)
    ratios2 = np.empty(shape=[0, 3])
    lonr = []
    latr = []
    for i in range(equal_area_points.latitude.shape[0]):
        ratios = np.vstack([A_bin_counts[i],B_bin_counts[i],C_bin_counts[i]]).T
        if np.any(ratios>0):
            ratios2 = np.append(ratios2, ratios, axis=0)
            lonr.append(equal_area_points.longitude[i])
            latr.append(equal_area_points.latitude[i])
    fig.plot(x=lonr, y=latr, pen='white', style="c0.11c")
    wedges_to_gmt2(lonr,latr,ratios2,['darkred','darkblue','darkgreen'],['A','B','C'])

    fig.basemap(region=region, projection=projection, frame=frame)
    fig.basemap(region=[0, 8, 0, 4], projection='X8c/4c', frame=True, transparency="100")
    fig.text(x=0.4, y=3.6, text=str(int(reconstruction_time))+" Ma", font=["7p,Helvetica",'black'],justify=["LM"])
    fig.text(text=['Supercontinental', 'Supercontinental'], x=[1.5, 3.8], y=[1.5, 1.8], font=["5p,Helvetica",'black'],justify=["CB"])
    fig.text(text=['margin', 'core'], x=[1.5, 3.8], y=[1.3, 1.6], font=["5p,Helvetica",'black'],justify=["CB"])

    vector_1 = [2.2, 1.5, -18, 1.1]
    vectors = [vector_1]
    fig.plot(data=vectors,style="v0.2c+eA",pen="0.5p",color="black")

    fig.shift_origin(xshift='-8.1c', yshift='-4.1c')
    fig.basemap(region=region, projection=projection, frame=frame)
    reconstruction_time = 250.

    reconstructed_continents = Y2019.polygon_snapshot('continents', reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_plates = Y2019.plate_snapshot(reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_continents.plot(fig, color='Gainsboro',pen='gray')

    A_bin_counts = get_binned_distances(partitioned_A, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    B_bin_counts = get_binned_distances(partitioned_B, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    C_bin_counts = get_binned_distances(partitioned_C, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)

    fig.plot(data="data/additional_pologys/polygon_2501_red.gmt",color=pattern_color1, pen=pen_color1, transparency=pattern_t)
    fig.plot(data="data/additional_pologys/polygon_2502_green.gmt",color=pattern_color2, pen=pen_color2, transparency=pattern_t)

    ratios2 = np.empty(shape=[0, 3])
    lonr = []
    latr = []
    for i in range(equal_area_points.latitude.shape[0]):
        ratios = np.vstack([A_bin_counts[i],B_bin_counts[i],C_bin_counts[i]]).T
        if np.any(ratios>0):
            ratios2 = np.append(ratios2, ratios, axis=0)
            lonr.append(equal_area_points.longitude[i])
            latr.append(equal_area_points.latitude[i])
    fig.plot(x=lonr, y=latr, pen='white', style="c0.11c")
    wedges_to_gmt2(lonr,latr,ratios2,['darkred','darkblue','darkgreen'],['A','B','C'])

    fig.basemap(region=region, projection=projection, frame=frame)
    fig.basemap(region=[0, 8, 0, 4], projection='X8c/4c', frame=True, transparency="100")
    fig.text(x=0.4, y=3.6, text=str(int(reconstruction_time))+" Ma", font=["7p,Helvetica",'black'],justify=["LM"])
    fig.text(text=['Supercontinental', 'Supercontinental'], x=[1.5, 4.2], y=[1.5, 1.8], font=["5p,Helvetica",'black'],justify=["CB"])
    fig.text(text=['margin', 'core'], x=[1.5, 4.2], y=[1.3, 1.6], font=["5p,Helvetica",'black'],justify=["CB"])

    vector_1 = [2.2, 1.5, 11, 1]
    vectors = [vector_1]
    fig.plot(data=vectors,style="v0.2c+eA",pen="0.5p",color="black")


    fig.shift_origin(xshift='8.1c')
    fig.basemap(region=region, projection=projection, frame=frame)
    anchor_plate_id = 201
    reconstruction_time = 350.

    reconstructed_continents = Y2019.polygon_snapshot('continents', reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_plates = Y2019.plate_snapshot(reconstruction_time, anchor_plate_id=anchor_plate_id)
    reconstructed_continents.plot(fig, color='Gainsboro',pen='gray')

    A_bin_counts = get_binned_distances(partitioned_A, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    B_bin_counts = get_binned_distances(partitioned_B, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)
    C_bin_counts = get_binned_distances(partitioned_C, reconstruction_model, reconstruction_time, anchor_plate_id=anchor_plate_id)

    fig.plot(data="data/additional_pologys/polygon_3501_green.gmt",color=pattern_color2, pen=pen_color2, transparency=pattern_t)
    #fig.plot(data="data/additional_pologys/polygon_3502_black.gmt",color="black", pen="0.5p,black", transparency=80)
    #fig.plot(data="data/additional_pologys/polygon_3503_black.gmt",color="black", pen="0.5p,black", transparency=80)
    fig.plot(data="data/additional_pologys/polygon_3504_red.gmt",color=pattern_color1, pen=pen_color1, transparency=pattern_t)
    fig.plot(data="data/additional_pologys/polygon_3505_red.gmt",color=pattern_color1, pen=pen_color1, transparency=pattern_t)

    ratios2 = np.empty(shape=[0, 3])
    lonr = []
    latr = []
    for i in range(equal_area_points.latitude.shape[0]):
        ratios = np.vstack([A_bin_counts[i],B_bin_counts[i],C_bin_counts[i]]).T
        if np.any(ratios>0):
            ratios2 = np.append(ratios2, ratios, axis=0)
            lonr.append(equal_area_points.longitude[i])
            latr.append(equal_area_points.latitude[i])
    fig.plot(x=lonr, y=latr, pen='white', style="c0.11c")
    wedges_to_gmt2(lonr,latr,ratios2,['darkred','darkblue','darkgreen'],['A','B','C'])

    fig.basemap(region=region, projection=projection, frame=frame)
    fig.basemap(region=[0, 8, 0, 4], projection='X8c/4c', frame=True, transparency="100")
    fig.text(x=0.4, y=3.6, text=str(int(reconstruction_time))+" Ma", font=["7p,Helvetica",'black'],justify=["LM"])
    fig.text(text=['Supercontinental', 'Supercontinental'], x=[1.2, 3.8], y=[1.5, 1.5], font=["5p,Helvetica",'black'],justify=["CB"])
    fig.text(text=['margin', 'core'], x=[1.2, 3.8], y=[1.3, 1.3], font=["5p,Helvetica",'black'],justify=["CB"])

    vector_1 = [1.9, 1.4, 25, 0.6]
    vector_2 = [1.9, 1.4, -29, 2.5]
    vectors = [vector_1, vector_2]
    fig.plot(data=vectors,style="v0.2c+eA",pen="0.5p",color="black")

    return fig


In [10]:
fig = pygmt.Figure()
fig6(pattern_color1='red', pattern_color2='green', pen_color1="0.5p,red", pen_color2="0.5p,green", pattern_t=80)
fig.savefig('plots/figures/Figure6.pdf', dpi=1000)
fig.savefig('plots/figures/Figure6.png', dpi=1000)

C:\Users\84746\anaconda3\envs\pygmt\lib\site-packages\pygmt\helpers\decorators.py:872: SyntaxWarning: Short-form parameter (t) is not recommended. Use long-form parameter 'transparency' instead.
  return module_func(*args, **kwargs)
